## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-08-05-49-29 +0000,nyt,"Trump Hosts the Kennedy Center Honors, Breakin...",https://www.nytimes.com/2025/12/07/us/politics...
1,2025-12-08-05-31-26 +0000,startribune,"Bob Dylan returning to home state, will perfor...",https://www.startribune.com/bob-dylan-returnin...
2,2025-12-08-05-30-38 +0000,nyt,8 Matisse Works Stolen From Library in Brazil,https://www.nytimes.com/2025/12/08/world/ameri...
3,2025-12-08-05-11-00 +0000,wsj,China’s Exports Bounce Back in Sign of Tariff ...,https://www.wsj.com/economy/trade/chinas-expor...
4,2025-12-08-05-08-17 +0000,nyt,The New Syria,https://www.nytimes.com/2025/12/07/world/syria...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2375/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,28
25,new,14
104,fire,12
34,he,10
10,will,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
30,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...,66
67,2025-12-07-22-30-00 +0000,wsj,President Trump criticized as disloyal an indi...,https://www.wsj.com/politics/elections/trump-l...,62
146,2025-12-07-13-42-20 +0000,cbc,"Israel's Netanyahu says he will meet Trump, se...",https://www.cbc.ca/news/world/netanyahu-says-h...,58
110,2025-12-07-19-42-14 +0000,cbc,Trump's new security strategy aligns with Russ...,https://www.cbc.ca/news/world/russia-trump-nat...,55
10,2025-12-08-04-49-10 +0000,bbc,Trump says $72bn Netflix-Warner Bros deal 'cou...,https://www.bbc.com/news/articles/cn815egjqjpo...,52


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
30,66,2025-12-08-02-40-46 +0000,wapo,Trump says he will be ‘involved’ in vetting Ne...,https://www.washingtonpost.com/business/2025/1...
93,38,2025-12-07-20-50-26 +0000,nypost,"Zohran Mamdani gives advice to thwart, evade I...",https://nypost.com/2025/12/07/us-news/zohran-m...
187,34,2025-12-07-08-36-56 +0000,nypost,"At least 25 dead, including tourists, in fire ...",https://nypost.com/2025/12/07/world-news/india...
36,32,2025-12-08-01-54-25 +0000,nypost,JPMorgan Chase’s Jamie Dimon claims AI will no...,https://nypost.com/2025/12/07/business/jamie-d...
139,32,2025-12-07-15-32-36 +0000,nypost,New theory reveals the disgusting reason behin...,https://nypost.com/2025/12/07/world-news/new-t...
89,29,2025-12-07-21-18-37 +0000,nypost,Food influencer Michael Duarte shared about pa...,https://nypost.com/2025/12/07/us-news/food-inf...
90,27,2025-12-07-21-16-54 +0000,nypost,Female customers at ritzy NYC eatery report bl...,https://nypost.com/2025/12/07/us-news/female-c...
182,26,2025-12-07-09-21-41 +0000,nypost,Soldiers on Benin’s national television claim ...,https://nypost.com/2025/12/07/world-news/benin...
74,26,2025-12-07-21-46-24 +0000,nypost,Holiday parade in California’s wealthy ‘Americ...,https://nypost.com/2025/12/07/us-news/holiday-...
20,24,2025-12-08-03-44-08 +0000,nyt,Hong Kong Holds Vote as Officials Move Against...,https://www.nytimes.com/2025/12/07/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
